Get injury data from http://www.foxsports.com/nfl/injuries.

    'seasons' : range(2013, 2018)
    'weeks' : range(1, 18) 
    'columns' : ['name', 'injury', 'status', 'position']
    
Save to format data/1/fs_injury_{year1-year2}.csv

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

%run ../PandaSoup/PandaSoup.ipynb

You imported PandaSoup!


In [3]:
import requests
response = requests.get("http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=10")
soup = BeautifulSoup(response.text, "lxml")

In [19]:
options = soup.find_all("option", selected=True)
print(options[0].text)
print(options[1].text.strip("Week "))
#print(soup.find(xpath="//section[1]/div/section[1]/div/div[2]/div/div[1]/select[1]/option[5]"))

2015
10


In [20]:
def extract_data(soup):
    players = {}
    table = soup.find_all(class_=re.compile('wisbb_fullPlayer'))
    options = soup.find_all("option", selected=True)
    season = options[0].text
    week = options[1].text.strip("Week ")
    for i, link in enumerate(table):
        row = link.findParent().findChildren('td')
        row = [td.text.strip() for td in row]
        # parse out name and position
        name, pos = (re.sub(r'\n\n.*\n', '', row[0]).split('\n'))
        if pos in ['WB', 'RB', 'TE']:
            row[0] = name
            row.append(pos)
            row.append(season)
            row.append(week)
            players[i] = row
    return pd.DataFrame(players).T

In [25]:
pages = []
for y in [2015]:
    for w in range(1, 18):
        pages.append((y, w))

In [ ]:
import copy
params2 = {
    'base' : 'http://www.foxsports.com/nfl/injuries?season={}&seasonType=1&week={}',
    'param_names' : ['season', 'week'],
    'request_delay' : 2,
    'extract_func': extract_data,
    'csv_base' : "data/1/fs_injury_{}.csv",
    'grouping_strategy' : ["season"]
}

ps2 = PandaSoup(params2)
ps2.scrape_all(pages, verbose=True)
len(ps2.raw_data)

parameter values (2015, 1) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=1
parameter values (2015, 2) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=2
parameter values (2015, 3) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=3
parameter values (2015, 4) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=4
parameter values (2015, 5) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=5
parameter values (2015, 6) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=6
parameter values (2015, 7) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=7
parameter values (2015, 8) --> key (2015,)
Reading http://www.foxsports.com/nfl/injuries?season=2015&seasonType=1&week=8
parameter values (2015, 9) --> k

In [23]:
ps2.make_dataframes()

defaultdict(list,
            {(2014,):                       0                            1                      2  \
             0      Bostick, Brandon                       fibula                    Out   
             4        Helfet, Cooper                         knee                    Out   
             8    Michael, Christine                    hamstring                    Out   
             14          Lorig, Erik                        ankle                    Out   
             19        Burkhead, Rex                         knee                    Out   
             21        Eifert, Tyler                     shoulder               Probable   
             23      Peerman, Cedric                          hip               Probable   
             28      Pierce, Bernard                   concussion               Probable   
             30        Dixon, Boobie                    hamstring           Questionable   
             34           Smith, Lee                 

In [24]:
ps2.to_csv()

['data/1/fs_injury_2014.csv']